In [11]:
import json
from glob import glob

import numpy as np
from tedana import decay, io

In [32]:
in_files = sorted(
    glob('/Users/taylor/Documents/datasets/nibs/dset/sub-02/ses-01/anat/*dir-AP*MESE.nii.gz')
)
echo_times = []
for f in in_files:
    json_file = f.replace('.nii.gz', '.json')
    with open(json_file, 'r') as fo:
        metadata = json.load(fo)
    te = metadata['EchoTime'] * 1000
    echo_times.append(te)

print('\n'.join(in_files))

data_cat, ref_img = io.load_data(in_files, n_echos=len(echo_times))

mask = np.ones(data_cat.shape[0], dtype=int)
masksum = mask * len(echo_times)

t2s_limited, s0_limited, t2s_full, s0_full = decay.fit_monoexponential(
    data_cat=data_cat,
    echo_times=echo_times,
    adaptive_mask=masksum,
    report=False,
)

/Users/taylor/Documents/datasets/nibs/dset/sub-02/ses-01/anat/sub-02_ses-01_dir-AP_run-01_echo-1_MESE.nii.gz
/Users/taylor/Documents/datasets/nibs/dset/sub-02/ses-01/anat/sub-02_ses-01_dir-AP_run-01_echo-2_MESE.nii.gz
/Users/taylor/Documents/datasets/nibs/dset/sub-02/ses-01/anat/sub-02_ses-01_dir-AP_run-01_echo-3_MESE.nii.gz
/Users/taylor/Documents/datasets/nibs/dset/sub-02/ses-01/anat/sub-02_ses-01_dir-AP_run-01_echo-4_MESE.nii.gz


/Users/taylor/Documents/tsalo/tedana/tedana/decay.py:280: RuntimeWarning: divide by zero encountered in divide
  t2s = 1.0 / betas[1, :].T


4-echo monoexponential:   0%|          | 0/3145728 [00:00<?, ?it/s]

/opt/miniconda3/envs/salo/lib/python3.9/site-packages/scipy/optimize/_lsq/common.py:424: RuntimeWarning: invalid value encountered in subtract
  upper_dist = ub - x
/opt/miniconda3/envs/salo/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:57: RuntimeWarning: invalid value encountered in subtract
  upper_dist = ub - x0
/opt/miniconda3/envs/salo/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:575: RuntimeWarning: invalid value encountered in scalar subtract
  dx = x[i] - x0[i]  # Recompute dx as exactly representable number.


KeyboardInterrupt: 

In [ ]:
t2s_s = t2s_limited / 1000
t2s_s[np.isinf(t2s_s)] = 0.5
s0_limited[np.isinf(s0_limited)] = 0

In [ ]:
t2s_s_img = io.new_nii_like(ref_img, t2s_s)
s0_img = io.new_nii_like(ref_img, s0_limited)
t2s_s_img.to_filename('T2starmap2.nii.gz')
s0_img.to_filename('S0map2.nii.gz')

In [30]:
t2s_s

array([ 0.5       ,  0.5       ,  0.5       , ...,  2.28056701,
        0.62559854, -3.09099682])

In [31]:
echo_times

[15.0, 30.0, 50.0, 100.0]

In [20]:
masksum

array([4, 4, 4, ..., 4, 4, 4])